# 快速實作
    做一個 Chatbot server, 當用戶傳照片給我們的時候，回傳 "哈哈哈 給客戶"

In [1]:
# 引用套件
from flask import (
    Flask, request, abort
)

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage
)

In [2]:
# 載入設定檔
import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定 Server 啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動 server 對外接口，使 Line 能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [3]:
# 消息素材準備
text_reply_message1 = TextSendMessage(text="哈哈哈 暈了")

In [4]:
# 業務邏輯處理
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        text_reply_message1
    )
    
    # 讓 line_bot_api 跟 line 要 圖片 回來
    message_content = line_bot_api.get_message_content(event.message.id)
    
    # 並把 圖片 儲存起來
    with open('../images/'+event.message.id + '.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [5]:
# 啟用伺服器
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2020 16:19:39] "POST / HTTP/1.1" 200 -
